## Linear Regression


Sklearn based models are slightly finicky to get into a suitable onnx format. 
This notebook showcases how to do so using the `hummingbird-ml` python package ! 

In [26]:
!pip install ezkl
!pip install hummingbird.ml
!pip install onnx

You should consider upgrading via the '/Users/ethancemer/Documents/development/ezkl/.env/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ethancemer/Documents/development/ezkl/.env/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "hummingbird-ml"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import torch
import ezkl
import json
from hummingbird.ml import convert


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)

circuit = convert(reg, "torch", X[:1]).model

In [2]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [3]:


# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

# Input to the model
shape = X.shape[1:]
# read in ./input_json
data = json.load(open("input.json", 'r'))
# convert to torch tensor
x = torch.tensor(data['input_data'], requires_grad=True)
torch_out = circuit(x)
# Export the model
torch.onnx.export(circuit,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [4]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True

In [6]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [7]:
# srs path
res = ezkl.get_srs(settings_path)

In [8]:
# now generate the witness file 

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [9]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

spawning module 2
spawning module 2


In [25]:
# GENERATE A PROOF
import time


proof_path = os.path.join('test.pf')
# log time it takes to generate proof
start = time.time()
res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )
end = time.time()
print("PROOF GENERATION TIME: ", end - start)

print(res)
print(res['instances'])
assert os.path.isfile(proof_path)

spawning module 2


PROOF GENERATION TIME:  0.03862285614013672
{'instances': [[[8514802640502574666, 4537455015751700300, 13297884588498018886, 498217386297169707]]], 'proof': '073a816d017051f247a3c263c0aef684f321e49cac18ecfb07563de119efdbc5276d03594dc22c0b364fb14256e4c55d608c23d4e7da397b9eda49b1ad81b7471533833374c40ace9d633df6a7079b7f958f05a9570898898c671375df02b94f06706f2d4cafc3ebe8eae223d877e04629546f57893e15ee4c10af9654b7cc2d28e28fb9e44824f895ab7c53c5e67b1cad04e4d7ec25f30097c1ac290bc46e7c1022f4188ec4b27c1102d0fa5a73829e5cf20bc1af046077c2aef422a2b8d6e80f448aa615aa2a458da9d64f618f9b0a3d6f8650959f907232c0bdc4108a64c50b7d47bd41865a29bc6c621888fbf64730881097210b5a8d110d97c512b98ca60da5bb9135730af04d930dbf4a6474fb4c3bb4d748d49d34ef1c910de59e25590b80048104d99a1354d6664bf4ad82ea4f41158ad454e5534f940efd6f269a4d11c6af85001e4c107b421f7a9a97d8facc7eaf8ff76c3da49765274cf0edab991970ea0cbbed4007c188509a3379ee3e304888a6a228b3bdf89413fe80d3bcf1133c4b31d8aa0621459f0a605e969c0b910c43b2d261dfd4af147127f8aeb4a212bd78d842

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")